## Tags objects with tensions and writes back tagged_objects_tensions

reads objects, adds the columns below, and writes back to `object_tensions.parquet` Each of the following columns are of type boolean

- `is_economic_labour_tension`,
- `is_sectarian_tension`,
- `is_environmental_tension`,
- `is_political_tension`,
- `is_service_related_tension`,
- `is_community_insecurity_tension`,
- `is_geopolitics_tension`,
- `is_intercommunity_relations_tension`

Currently uses the latest CountVectorizerTensionClassifier trained to predict which tensions are present in objects. It only classifies tensions it has trained for, with all other tensions being False.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import pickle

import pandas as pd
import tentaclio

from phoenix.common import artifacts, run_datetime
from phoenix.common import utils
from phoenix.custom_models.tension_classifier.tension_classifier import CountVectorizerTensionClassifier

In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

# Filters for batch
YEAR_FILTER = 2021
# Without zero padding
MONTH_FILTER = 8
OBJECT_TYPE = "facebook_posts"

# Model URLs
MODEL_SUFFIX = ""
MODEL_BASE_URL = None

In [ ]:
if RUN_DATETIME:
    run_dt = run_datetime.from_file_safe_str(RUN_DATETIME)
else:
    run_dt = run_datetime.create_run_datetime_now()
    
if not MODEL_BASE_URL:
    MODEL_BASE_URL = f"{artifacts.urls.get_local()}{run_dt.to_run_date_str()}/"
    
url_config = {
    "YEAR_FILTER": YEAR_FILTER,
    "MONTH_FILTER": MONTH_FILTER,
    "OBJECT_TYPE": OBJECT_TYPE,
}
art_url_reg = artifacts.registry.ArtifactURLRegistry(run_dt, ARTIFACTS_ENVIRONMENT_KEY, artifacts.registry_mappers.get_default_mappers())
TAGGING_RUNS_URL_PIPELINE_BASE = art_url_reg.get_url("tagging_runs-pipeline_base", url_config)
TAGGING_RUNS_URL_OBJECTS = art_url_reg.get_url("tagging_runs-objects", url_config)
TAGGING_RUNS_URL_TOPICS = art_url_reg.get_url("tagging_runs-topics", url_config)
TAGGING_RUNS_URL_OBJECTS_TENSIONS = art_url_reg.get_url("tagging_runs-objects_tensions", url_config)

In [ ]:
# Display params.
print(
TAGGING_RUNS_URL_PIPELINE_BASE,
TAGGING_RUNS_URL_OBJECTS,
TAGGING_RUNS_URL_TOPICS,
TAGGING_RUNS_URL_OBJECTS_TENSIONS,
MODEL_BASE_URL,
run_dt.dt,
sep='\n',
)

In [ ]:
objects_df = artifacts.dataframes.get(TAGGING_RUNS_URL_OBJECTS).dataframe

In [ ]:
objects_df

In [ ]:
# will later use this, but not yet
# topics_df = artifacts.dataframes.get(TAGGING_RUNS_URL_TOPICS)

In [ ]:
# Model Suffix can later be used to distinguish between models, but for now is left empty
# and we use the latest model.
model_filename = CountVectorizerTensionClassifier.get_model_name(MODEL_SUFFIX)

In [ ]:
filepath_model = f"{MODEL_BASE_URL}{model_filename}.pickle"

In [ ]:
with tentaclio.open(filepath_model, "rb") as f:
    tension_classifier = pickle.load(f)

In [ ]:
print(f"We will classify the following tensions: {tension_classifier.class_labels}")

In [ ]:
objects_df = tension_classifier.predict(objects_df, "clean_text")

In [ ]:
objects_df[tensions_cols] = objects_df[tensions_cols].astype(bool) 

In [ ]:
a = artifacts.dataframes.persist(TAGGING_RUNS_URL_OBJECTS_TENSIONS, objects_df)